In [2]:
import pandas as pd
import numpy as np

# Update air visits

In [3]:
air_visits = pd.read_csv('../input/air_visit_data.csv')
air_reserve = pd.read_csv('../input/air_reserve.csv')
air_store_info = pd.read_csv('../input/air_store_info.csv')
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
sample_submission = pd.read_csv('../input/sample_submission.csv')

In [4]:
wkend_holidays = date_info.apply(lambda x: (x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1, axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5 

In [5]:
visit_data = air_visits.merge(date_info, on='visit_date', how='left')
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

In [7]:
for i in range(6,len(visit_data)):
    try:
        dow = visit_data.iloc[i]['day_of_week']
        flg = visit_data.iloc[i]['holiday_flg']
        store_id = visit_data.iloc[i]['air_store_id']
        top = visit_data[:i]
        top = top[(top['day_of_week'] == dow) & (top['holiday_flg'] == flg) & (top['air_store_id'] == store_id)]
        visitors = top.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() ))[0]
        visit_data.at[i, 'weighted_visits'] = visitors
    except:
        pass
    if i % 1000 == 0:
        print (i)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [8]:
visit_data.to_csv('new_visit.csv', index=False)

# Deal with missing values

In [54]:
visit_data = pd.read_csv('new_visit.csv')
date_info = pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
visit_data.drop(['day_of_week', 'holiday_flg', 'weight'], axis=1, inplace= True)

In [55]:
wkend_holidays = date_info.apply(lambda x: (x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1, axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((517 - date_info.index) / len(date_info)) ** 5

In [56]:
visit_data = visit_data.merge(date_info, on='visit_date', how='left')

In [57]:
for i in range(0,len(visit_data)):
    if pd.isnull(visit_data.iloc[i]['weighted_visits']):
        try:
            dow = visit_data.iloc[i]['day_of_week']
            flg = visit_data.iloc[i]['holiday_flg']
            store_id = visit_data.iloc[i]['air_store_id']
            bottom = visit_data[i+1:]
            bottom = bottom[(bottom['day_of_week'] == dow) & (bottom['holiday_flg'] == flg) & (bottom['air_store_id'] == store_id)]
            visitors = bottom.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() ))[0]
            visit_data.at[i, 'weighted_visits'] = visitors
        except:
            pass
    if i % 1000 == 0:
        print (i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [63]:
group = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg'], as_index=False)[['weighted_visits']].mean()

In [67]:
# fill missings with (air_store_id, day_of_week)
missings = visit_data.weighted_visits.isnull()
visit_data.loc[missings, 'weighted_visits'] = visit_data[missings].merge(
    group[group.holiday_flg==0], on=('air_store_id', 'day_of_week'), how='left')['weighted_visits_y'].values

In [70]:
missings = visit_data.weighted_visits.isnull()
visit_data.loc[missings, 'weighted_visits'] = visit_data[missings].merge(
    group[['air_store_id', 'weighted_visits']].groupby('air_store_id').mean().reset_index(), on='air_store_id', how='left')['weighted_visits_y'].values

In [72]:
visit_data['weighted_visits'] = visit_data.weighted_visits.map(pd.np.expm1)

In [74]:
visit_data.to_csv('weight_visit.csv', index=False)

# Update sample submission

### use hklee result

# merge with train val sub

In [77]:
# read all the data
train = pd.read_csv('../train/training_data2.csv')
val = pd.read_csv('../train/validation_data2.csv')
test = pd.read_csv('../train/submission_with_features2.csv')

past_air_visit = pd.read_csv('weight_visit.csv')
past_visits_sub = pd.read_csv('sub_weight_visits.csv')

In [88]:
past_air_visit.drop(['day_of_week', 'holiday_flg', 'visitors','weight'], axis=1, inplace= True)

In [85]:
visit_data2 = pd.read_csv('new_visit.csv')
visit_data2.drop(['day_of_week', 'holiday_flg', 'visitors','weighted_visits'], axis=1, inplace= True)

In [90]:
past_air_visit = pd.merge(past_air_visit, visit_data2, how = 'left', on = ['air_store_id', 'visit_date'])

In [92]:
train = pd.merge(train, past_air_visit, how = 'left', on = ['air_store_id', 'visit_date'])
val = pd.merge(val, past_air_visit, how = 'left', on = ['air_store_id', 'visit_date'])

In [96]:
past_visits_sub = past_visits_sub.rename(columns={'visitors':'weighted_visits'})

In [97]:
test = pd.merge(test, past_visits_sub, how = 'left', on = ['id'])

In [99]:
train.to_csv('training_data3.csv', index=False)
val.to_csv('validation_data3.csv', index=False)
test.to_csv('submission_with_features3.csv', index=False)